### Import Library yang digunakan

In [983]:
import numpy.random as npRandom
import numpy as np
import itertools
import random
import csv
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [984]:
import pandas as pd
dataset = pd.read_csv("dataSet.txt")

### Generate Kromosom Biner

In [985]:
# Digunakan untuk meng generate kromosom menjadi biner
def generateKromosom(jumlahPopulasi, panjangKromosom):
    populasi = [[npRandom.randint(0,2) 
    for i in range(panjangKromosom)] 
    for j in range(jumlahPopulasi)]
    return populasi

## Encode label jadi biner

In [986]:
# Melakukan encode menjadi angka
def encodeToInteger(data):
    values = array(data)
    # integer encode
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(values)
    return integer_encoded

#Encode Label menjadi biner
def encodeLabel(data):
    values = array(data)
    # integer encode
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(values)
    # binary encode
    onehot_encoder = OneHotEncoder(sparse=False)
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
    # invert first example
    inverted = label_encoder.inverse_transform([argmax(onehot_encoded[0, :])])

    onehot_encoded = onehot_encoded.astype(int)
    return onehot_encoded.tolist()

# def changeLabelToArray(dataSet):
#     suhu = dataset[['Suhu']]
#     waktu = dataset[['Waktu']]
#     kondisi_langit = dataset[['Kondisi Langit']]
#     kelembapan = dataset[['Kelembapan']]
#     terbang = dataset[['Terbang/Tidak']]
    
#     arrSuhu = encodeLabel(suhu)
#     arrWaktu = encodeLabel(waktu)
#     arrKondisiLangit = encodeLabel(kondisi_langit)
#     arrKelembapan = encodeLabel(kelembapan)
#     arrTerbang = encodeLabelTerbang(terbang)
# #     arrTerbang = encodeLabel(terbang)
#     listDataSet = []
#     for i in range(0, len(suhu)):
# #         tmpKromosom1 = arrSuhu[i]+arrWaktu[i]+arrKondisiLangit[i]+arrKelembapan[i]+arrTerbang[i]
#         tmpKromosom1 = arrSuhu[i]+arrWaktu[i]+arrKondisiLangit[i]+arrKelembapan[i]+[arrTerbang[i]]
#         listDataSet.append(tmpKromosom1)
#     return listDataSet

def changeLabelToArray(data):
    arrSuhu = encodeLabel(data[['Suhu']])
    arrWaktu = encodeLabel(data[['Waktu']])
    arrKondisiLangit = encodeLabel(data[['Kondisi Langit']])
    arrKelembapan = encodeLabel(data[['Kelembapan']])
    arrTerbang = encodeToInteger(data[['Terbang/Tidak']])
    listDataSet = []
    for i in range(0, len(arrSuhu)):
        tmpKromosom = arrSuhu[i]+arrWaktu[i]+arrKondisiLangit[i]+arrKelembapan[i]+[arrTerbang[i]]
        listDataSet.append(tmpKromosom)
    return listDataSet

### Nilai Fitness

In [987]:
def cekBenar(indexBawah, indexAtas, indexBawahData, kromosom, dataSet):
    Benar = False
    z = indexBawahData
    for j in range(indexBawah, indexAtas):
        if(dataSet[z] == 1 and kromosom[j] == 1):
            Benar = True
            break;
        z += 1
    return Benar

def cekBenarTerbang(indexAtas, kromosom, dataSet):
    Benar = False
    if(dataSet[14] == kromosom[indexAtas]):
            Benar = True
    return Benar

def nilaiFitness(kromosom, dataSet):
    nilai = 0
    nAturan = round(len(kromosom)/len(dataSet[0]))
    for i in range(0, len(dataSet)):
        for j in range(0, nAturan):
            tmp = j * len(dataSet[0])
            cekWaktu = cekBenar(0+tmp, 3+tmp,0, kromosom, dataSet[i])
            cekSuhu = cekBenar(3+tmp, 7+tmp,3, kromosom, dataSet[i])
            cekKondisiLangit = cekBenar(7+tmp, 11+tmp,7, kromosom, dataSet[i])
            cekKelembapan = cekBenar(11+tmp, 14+tmp,11, kromosom, dataSet[i])
            cekTerbang = cekBenarTerbang(14+tmp, kromosom, dataSet[i])
            if(cekWaktu and cekSuhu and cekKondisiLangit and cekKelembapan and cekTerbang):
                nilai += 1
                break
    return (nilai / len(dataSet)) * 100

## Pemilihan Orang Tua (Tournament Selection)

In [988]:
def tournamentSelection(populasi, panjangTournament):
    idxSample = np.arange(len(populasi))
    npRandom.shuffle(idxSample)
    idxSample = idxSample[0:panjangTournament]
    fitnesses = [nilaiFitness(populasi[idxSample[i]], dataSet) for i in range(0,panjangTournament)]
    mergedArray = [(idxSample[i], fitnesses[i]) for i in range(panjangTournament)]
    mergedArray = sorted(mergedArray, key = lambda a: a[1], reverse = True)
    return mergedArray[0][0], mergedArray[1][0]

## Crossover (Only Increasing)

In [989]:
def crossover(parent1, parent2, pC, panjangAturan):
    prob = npRandom.random()
    anak1 = parent1[:]
    anak2 = parent2[:]
    if(prob <= pC):
        #Jika panjang parent 1 > parent 2 maka ditukar agar mudah dikodingan bawahnya
        if(len(parent1)>len(parent2)):
            tmpParent = parent1[:]
            parent1 = parent2[:]
            parent2 = tmpParent[:]
        
        #Random titik untuk parent 1
        pointParent1 = [npRandom.randint(1,len(parent1)-1) for i in range(0,2)]
        pointParent1.sort()
        pointParent2 = []
        
        #Buat kemungkinan titik yang bisa dibangun untuk parent2
        gap1 = pointParent1[1] - pointParent1[0]
        gap2 = gap1 % panjangAturan
        pointParent2.append([pointParent1[0]]+[pointParent1[0]+gap1])
        pointParent2.append([pointParent1[0]]+[pointParent1[0]+gap2])
        pointParent2.append([pointParent1[1]-gap1]+[pointParent1[1]])
        pointParent2.append([pointParent1[1]-gap2]+[pointParent1[1]])
    
        #Hilangkan list yang sama dalam list parent2
        pointParent2.sort()
        pointParent2 = list(pointParent2 for pointParent2,_ in itertools.groupby(pointParent2))
        
        #Pilih kemungkinan titik yang sudah dicari secara random
        pointParent2 = random.choice(pointParent2)
        
        if(pointParent1 == pointParent2):
            anak1 = parent1[0:pointParent1[0]]+parent2[pointParent1[0]:pointParent1[1]]+parent1[pointParent1[1]:len(parent1)]
            anak2 = parent2[0:pointParent1[0]]+parent1[pointParent1[0]:pointParent1[1]]+parent2[pointParent1[1]:len(parent1)]
        else:
            tmpPoint = (len(parent2)+panjangAturan)-((pointParent1[1]-pointParent1[0])+pointParent2[0])
            while(tmpPoint < 0):
                tmpPoint = tmpPoint + panjangAturan
            anak1 = parent1[0:pointParent1[0]]+parent2[pointParent1[0]:pointParent1[1]]+parent1[pointParent1[1]:len(parent1)]
            anak2 = parent2[0:pointParent2[0]]+parent1[pointParent1[0]:pointParent1[1]]+parent2[len(parent2)-tmpPoint:len(parent2)]
    return anak1,anak2

#Untuk hapus kromosom yang bit-nya 1 semua
def cekKromosom(kromosom, panjangAturan):
    nAturan = len(kromosom)/panjangAturan
    for i in range(0, round(nAturan)):
        if(kromosom[i*panjangAturan:(i*panjangAturan)+(panjangAturan-1)]==[1,1,1,1,1,1,1,1,1,1,1,1,1,1]):
            for j in range(0, panjangAturan):
                del kromosom[i*15]
            i -= 1
    return kromosom

### Mutasi Bit Level Mutation

In [990]:
def mutasi(kromosom, pM):
    for i in range(len(kromosom)):
        prob = npRandom.random()
        if(prob <= pM):
            tmp = kromosom[i]
            while(tmp==kromosom[i]):
                kromosom[i] = npRandom.randint(2)
    return kromosom

## Pergantian Generasi (Steady State : Fitness-Based Selection)

In [991]:
def steadyState(jumlahGeneration, populasi, jumlahPopulasi, panjangTournament):
    for z in range(jumlahGeneration):
        gabungan = []
        child = []
        fitnesses = []
        for i in range(round(len(populasi)/2)):
            idxParent1, idxParent2 = tournamentSelection(populasi, panjangTournament)
            anak1 = populasi[idxParent1]
            anak2 = populasi[idxParent2]
            
            #Crossover
            anak1,anak2 = crossover(anak1,anak2,pC,panjangAturan)
            
            #Mutasi
            anak1 = mutasi(anak1,pM)
            anak2 = mutasi(anak2,pM)
            
            #Cek jika ada kromosom yang gen-nya 1 bit semua
            anak1 = cekKromosom(anak1, panjangAturan)
            anak2 = cekKromosom(anak2, panjangAturan)
            
            child.append(anak1)
            child.append(anak2)
            
        gabungan = populasi + child
        for i in range(len(gabungan)):
            fitnesses.append(nilaiFitness(gabungan[i],dataSet))
        mergedArray = [(gabungan[i], fitnesses[i]) for i in range(len(gabungan))]
        mergedArray = sorted(mergedArray, key = lambda a : a[1], reverse = True)
        pop = []
        tmp = []
        t = 0
        jumlah = 0
        
        for i in range(len(gabungan)):
            if(i >= 1):
                if(mergedArray[i][1] == mergedArray[jumlah-1][1]):
                    tmp.append(mergedArray[i][0])
                    t += 1
                else:
                    pop.append(mergedArray[i][0])
                    jumlah+=1
            else:
                pop.append(mergedArray[i][0])
                jumlah+=1
            if(jumlah == jumlahPopulasi):
                break
        
        z = 0
        while(jumlah < len(populasi)):
            pop.append(tmp[z])
            z+=1
            jumlah+=1
        
        populasi = []
        populasi = pop
        
    return populasi

## Deklarasi Variable

In [992]:
jumlahPopulasi = 30
jumlahGeneration = 50
panjangAturan = 15
panjangTournament = 15
panjangKromosom = 30

pC = 0.65
pM = 0.065
        
dataSet = changeLabelToArray(dataset)
print(dataSet)

[[1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0], [1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0], [1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0], [1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1], [1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0], [1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1], [1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1], [1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0], [1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0], [1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1], [1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0], [1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0], [1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1], [1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1], [1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0], [1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0], [1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0], [1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0], [1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1], [1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0], [1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1], [1, 0, 0, 0,

C:\Users\Faisal Ridwan\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
C:\Users\Faisal Ridwan\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Faisal Ridwan\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer da

### Main

In [993]:
populasi = generateKromosom(jumlahPopulasi, panjangKromosom)
pop = steadyState(jumlahGeneration ,populasi, jumlahPopulasi, panjangTournament)

# pop[0] = [9,4,7, 4, 4, 8]
print("Kromosom : "+str(pop[0]))
nilai = nilaiFitness(pop[0],dataSet)
print("Nilai Fitness/akurasi : "+str(nilai))
print("Panjang kromosom "+str(len(pop[0])))

Kromosom : [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Nilai Fitness/akurasi : 100.0
Panjang kromosom 165


### Klasifikasi

In [994]:
dataUji = pd.read_csv("datauji.txt")

dataUji = changeLabelToArray(dataUji)
print(len(dataUji))
for i in range(0, len(dataUji)):
    dataUji[i] = dataUji[i][0:14]

print(len(dataUji))

def klasifikasi(dataUji, kromosom):
    arr = []
    nAturan = round(len(kromosom)/(len(dataUji)+1))
    for j in range(0, nAturan):
        tmp = j * (len(dataUji)+1)
        cekWaktu = cekBenar(0+tmp, 3+tmp,0, kromosom, dataUji)
        cekSuhu = cekBenar(3+tmp, 7+tmp,3, kromosom, dataUji)
        cekKondisiLangit = cekBenar(7+tmp, 11+tmp,7, kromosom, dataUji)
        cekKelembapan = cekBenar(11+tmp, 14+tmp,11, kromosom, dataUji)
        if(cekWaktu and cekSuhu and cekKondisiLangit and cekKelembapan):
#             print(tmp+15)
            if(kromosom[tmp]==0):
                
                return "0"
                break
            else:
                return "1"
                break
    return "0"

20
20


C:\Users\Faisal Ridwan\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
C:\Users\Faisal Ridwan\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Faisal Ridwan\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer da

In [995]:
   
arr = []
for i in range(0,len(dataUji)):
    arr.append(klasifikasi(dataUji[i],pop[0]))
print(arr)


['1', '1', '1', '0', '1', '1', '1', '1', '1', '1', '1', '1', '0', '1', '1', '1', '1', '0', '1', '0']


In [996]:
file = open('Data_Latih.csv','w',newline='\n')
tls = csv.writer(file)
tls.writerow(['Terbang/Tidak'])
for i in range(len(arr)):
    tls.writerow(arr[i])
file.close()